# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`
## Do Q2, and one of Q1 or Q3.

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?

This paper is about data tidying and provides a framework to make it easier to tidy messy datasets. The framework this paper provides also makes it easier to develop tidy tools for analysis.  

  2. Read the introduction. What is the "tidy data standard" intended to accomplish?

It is intended to facilitate an initial exploration and analysis of the data, and to simplify the development of data development tools that work well together.

  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."

The first quote means that tidy datasets provide a standardized approach to combine dataset structure (such as the number of rows and columns) with its meaning. Messy datasets, however, are messy in their own unique ways. Some are missing values, others have duplicate entries, etc.

The second quote means that it is difficult to provide a standardized definition of what observations and variables are.

  4. Read Section 2.2. How does Wickham define values, variables, and observations?

Values are either numbers or strings, organized in two ways: variables and observations.

Variables contain all values that measure the same underlying attribute (height, temperature, duration, etc.)

Observations contain all values measured on the same unit (person, day, race) across attributes.

  5. How is "Tidy Data" defined in section 2.3?

Tidy Data is defined as: 1) Each variable forms a column. 2) Each observation forms a row, 3) Each type of observational unit forms a table.

  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?

- Column headers are values, not variable names
- Multiple variables are stored in one column
- Variables are stored in both rows and columns
- Mulptiple types of observational units are stored in the same table
- A single observational unit is stored in multiple units


  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?

Table 11 is messy because it contains missing values in many of the cells. Table 12 fixes this by omitting missing values to save space, but the "element" column contains names instead of variables.

  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

If tidy tools is only as useful as the tools that work with it, then tidy tools will be inextricably linked to tidy data. Wickham hopes that long-term, concerted effort will be taken to break out of the "local maxima" where "independently changing data structures or data tools will not improve workflow". He hopes that others will build on the data tidying framework he has outliend to enhance data storage tools and strategies.

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)




2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

! git clone https://github.com/DS3001/wrangling

# Question 2.1
data = pd.read_csv('wrangling/assignment/data/airbnb_hw.csv', low_memory = False)
price = data['Price']
price = price.str.replace(',','') # replacing cells with just comma to blank cells
price = pd.to_numeric(price,errors='coerce') # converting string values to float/numeric
print('Number of missing cells: ', sum(price.isnull()))

fatal: destination path 'wrangling' already exists and is not an empty directory.
Number of missing cells:  0


In [23]:
# Question 2.2
data = pd.read_csv('wrangling/assignment/data/sharks.csv', low_memory = False)
data['Type'].value_counts()
type = data['Type']
type = type.replace(['Sea Disaster', 'Boat','Boating','Boatomg'],'Watercraft') # consolodiating all watercraft-related incidents
type.value_counts()
type = type.replace(['Invalid', 'Questionable','Unconfirmed','Unverified','Under investigation'],np.nan) # replace invalid data with NA
type.value_counts()

data['Type'] = type
del type
data['Type'].value_counts()

Unprovoked    4716
Provoked       593
Watercraft     583
Name: Type, dtype: int64

In [35]:
# Question 2.3
data = pd.read_csv('/content/wrangling/data/2017_pretrial.csv', low_memory=False)
released = data['WhetherDefendantWasReleasedPretrial']
print(released.unique(),'\n')
print(released.value_counts(),'\n')
released = released.replace(9,np.nan) # 9's are defined as "unclear"
sum(released.isnull()) # 26 missing values
data['WhetherDefendantWasReleasedPretrial'] = released
del released

[9 0 1] 

1    11907
0     3571
9       26
Name: WhetherDefendantWasReleasedPretrial, dtype: int64 



In [39]:
# Question 2.4
data = pd.read_csv('/content/wrangling/data/2017_pretrial.csv', low_memory=False)
len = data['ImposedSentenceAllChargeInContactEvent']
type = data['SentenceTypeAllChargesAtConvictionInContactEvent']
len = pd.to_numeric(len,errors='coerce') # converting all valoues to numeric
na_len = len.isnull() # dummy value
print( np.sum(na_len),'\n') # 5k missing values -- really bad
len = len.mask(type==4, 0)
len = len.mask(type==9, np.nan)
na_len = len.isnull() # another dummy
print( np.sum(na_len),'\n') # only 132
data['ImposedSentenceAllChargeInContactEvent'] = len # replace with cleaned version
del len, type

5492 

132 



**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?